# Chapter 11: Conditional Logic

There are use-cases where conditionals are requirement like an if-else.


In [1]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL
from sqlalchemy.orm import Session
import pandas as pd


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

# Case Expressions

Instead of using joins, list of customer names whether it is an individual or a business


In [2]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            c.cust_id
            , c.fed_id
            , (CASE c.cust_type_cd
                WHEN 'I' THEN (
                    SELECT
                        CONCAT('i-', i.fname, ' ', i.lname)
                    FROM individual i
                    WHERE c.cust_id = i.cust_id
                )
                WHEN 'B' THEN (
                    SELECT
                        CONCAT('b-', b.name)
                    FROM business b
                    WHERE b.cust_id = c.cust_id
                )
                ELSE 'UNKNOWN'
                END) name
        FROM
            customer c
        ;
        """,
        con=session.connection()
    )

print(df)

    cust_id       fed_id                      name
0         1  111-11-1111            i-James Hadley
1         2  222-22-2222           i-Susan Tingley
2         3  333-33-3333            i-Frank Tucker
3         4  444-44-4444            i-John Hayward
4         5  555-55-5555         i-Charles Frasier
5         6  666-66-6666            i-John Spencer
6         7  777-77-7777          i-Margaret Young
7         8  888-88-8888             i-Louis Blake
8         9  999-99-9999          i-Richard Farley
9        10   04-1111111     b-Chilton Engineering
10       11   04-2222222  b-Northeast Cooling Inc.
11       12   04-3333333      b-Superior Auto Body
12       13   04-4444444      b-AAA Insurance Inc.


# Selective Aggregation

Find all account whose account balances don't agree with the raw transaction data.

In [9]:
with Session(engine) as session:

    df = pd.read_sql_query(
        """
        SELECT
            CONCAT('ALERT! : Account #' , a.account_id, ' has incorrect balance') alert
        FROM
            account a
        WHERE (a.avail_balance, a.pending_balance) <> 
            (SELECT
                SUM(
                    CASE
                        WHEN t.funds_avail_date > CURRENT_TIMESTAMP()
                            THEN 0
                        WHEN t.txn_type_cd = 'DBT'
                            THEN -1 * t.amount
                        ELSE
                            t.amount
                    END
                ) avail
                , SUM(
                    IF(t.txn_type_cd = 'DBT', -1 * t.amount, t.amount)
                ) pend
            FROM transaction t
            WHERE t.account_id = a.account_id
        );
        """,
        con=session.connection()
    )

print(df)

                                         alert
0    ALERT! : Account #1 has incorrect balance
1    ALERT! : Account #2 has incorrect balance
2    ALERT! : Account #3 has incorrect balance
3    ALERT! : Account #4 has incorrect balance
4    ALERT! : Account #5 has incorrect balance
5    ALERT! : Account #7 has incorrect balance
6    ALERT! : Account #8 has incorrect balance
7   ALERT! : Account #10 has incorrect balance
8   ALERT! : Account #11 has incorrect balance
9   ALERT! : Account #12 has incorrect balance
10  ALERT! : Account #13 has incorrect balance
11  ALERT! : Account #14 has incorrect balance
12  ALERT! : Account #15 has incorrect balance
13  ALERT! : Account #17 has incorrect balance
14  ALERT! : Account #18 has incorrect balance
15  ALERT! : Account #19 has incorrect balance
16  ALERT! : Account #21 has incorrect balance
17  ALERT! : Account #22 has incorrect balance
18  ALERT! : Account #23 has incorrect balance
19  ALERT! : Account #24 has incorrect balance
20  ALERT! : 